In [1]:
import keras
import tensorflow as tf

# Memory Pre-configuration
config = tf.compat.v1.ConfigProto(
    gpu_options=tf.compat.v1.GPUOptions(
        per_process_gpu_memory_fraction=0.9,
        allow_growth = True
    )
    # device_count = {'GPU': 1}
)
session = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(session)

훈련하는 동안 모델 내부에서 일어나는 일을 조사하고 제어하는 방법을 살펴보겠습니다.
* 대규모 데이터셋에서 `model.fit()`이나 `model.fit_generator()`로 수십 번의 에포크를 실행하는 것은 **종이 비행기**를 날리는 것과 비슷합니다.
* 손을 떠나면 종이 비행기 경로와 착륙 지점을 제어할 방법이 없습니다
* 나쁜 결과를 피하려면 종이 비행기 대신 **드론**을 사용하는 것이 좋습니다
* 앞으로 소개할 기법은 `model.fit()` 호출을 종이 비행기에서 스스로 판단하고 동적으로 결정하는 똑똑한 자동 드론으로 바꾸어 줄 것입니다

### 콜백을 사용하여 모델의 훈련 과정 제어하기

모델을 훈련할때 미리 예상할 수 없는 것들이 많습니다.
* 최적의 검증 손실을 얻기 위해 얼마나 많은 에포크가 필요한지 알지 못합니다
* 지금까지는 적절한 훈련 에포크를 얻기 위해 첫번재 실행에서 **과대적합**이 시작될때까지 충분한 에포크로 훈련했습니다
* 그런 다음 최적의 에포크 횟수로 처음부터 새로운 훈련을 시작했습니다
* 이런 방식은 낭비가 많습니다.

더 좋은 처리 방식은 **검증 손실이 더 이상 향상되지 않을때** 훈련을 멈추는 것입니다.
* 케라스의 **콜백 (callback)** 사용하여 구현할 수 있습니다
* 콜백은 모델의 *fit()* 메서드가 호출될때 전달되는 객체입니다
* 훈련하는 동안 모델은 여러지점에서 콜백을 호출합니다
* 콜백은 모델의 상태와 성능에 대한 모든 정보를 접근하고 훈련 중지, 모델 저장, 가중치 적재 또는 모델 상태 변경등을 처리할 수 잇습니다.

다음은 콜백의 예제입니다
* **모델 체크포인트 저장**: 훈련하는 동안 어떤 지점에서 모델의 현재 가중치를 저장
* **조기 종료 (early stopping)**: 검증 손실이 더 이상 향상되지 않을대 훈련을 중지
* **하이퍼파라미터 값 동적으로 조정**: 옵티마이저의 학습률 같은 경우
* **시각화**

콜백 사용법을 익히기 위해 ModelCheckpoint, EarlyStopping, ReduceLROnPlateau를 사용한 예

#### ModelCheckpoint 와 EarlyStopping 콜백

**EarlyStopping** 콜백을 사용하면 정해진 에포크 동안 모니터링 지표가 향상되지 않을 때 훈련을 중지할 수 있습니다.
* 예를 들면 과대적합이 시작되자마자 훈련을 중지할 수 있습니다.
* 따라서 에포크 횟수를 줄여 다시 모델을 훈련할 필요가 없습니다.
* 일반적으로 이 콜백은 훈련하는 동안 모델을 계속 저장해주는 **ModelCheckPoint**와 함께 사용합니다.

```python
import keras

## fit() 메서드의 callbacks 매개변수를 사용하여 콜백의 리스트를 모델로 전달합니다
callbacks_list = [
    
    ## 성능 향상이 멈추면 훈련을 중지하는 콜백
    ## monitor = "val_acc" --> 모델의 검증 정확도를 모니터링
    ## patience = 1 --> 1 에포크보다 더 길게 (즉 2 에포크 동안) 정확도가 향상되지 않으면 훈련이 중지
    keras.callbacks.EarlyStopping(monitor = "val_acc", 
                                  patience = 1), 
    
    ## 에포크마다 현재 가중치를 저장
    ## filtepath = "my_model.h5" --> 모델의 파일 경로
    ## monitor = "val_loss" --> val_loss가 좋아지지 않으면 모델 파일을 덮어쓰지 않음
    keras.callbacks.ModelCheckpoint(filepath = "my_model.h5", 
                                    monitor = "val_loss", 
                                    save_best_only= = True)
]

model.compile(optimizer = "rmsprop",
              loss = "binary_crossentropy",
              metrics = ["acc"])

model.fit(x, y, 
          epochs = 10, 
          batch_size = 32, 
          callbacks = callback_list, 
          validation_data = (x_val, y_val))
```

#### ReduceLROnPlateau 콜백

이 콜백을 사용하면 *검증 손실이 향상되지 않을때 학습률을 작게 만들 수 있습니다*. 
* 학습률을 작게 하거나 크게 하면 훈련 도중 지역 최솟값에서 효과적으로 빠져나올 수 있습니다

```python
callbacks_list = [
    keras.callbacks.ReduceLROnPlateau(monitor = "val_loss", ## 모델의 검증 손실을 모니터링
                                      factor = 0.1, ## 콜백이 호출될때 학습률을 10배로 줄입니다
                                      patience = 10) ## 검증 소실이 10 에포크 동안 좋아지지 않으면 콜백이 호출됩니다
]

```

#### 자신만의 콜백 만들기

내장 콜백에서 제공하지 않은 특수한 행동이 훈련 도중 필요하면 자신만의 콜백을 만들 수 있습니다.

```python
on_epoch_begin ## 각 에포크가 시작할때 호출
on_epoch_end ## 각 에포크가 끝날때 호출

on_batch_begin ## 각 배치 처리가 시작되기전에 호출
on_batch_end ## 각 배치 처리가 끝날때 호출

on_train_begin ## 훈련이 시작될때 호출
on_train_end ## 훈련이 끝날때 호출
```

<br></br>
이 메서드들은 모두 logs 매개변수와 함께 호출됩니다. 이 매개변수는 이전 배치, 에포크에 대한 훈련과 검증 측정값이 담겨있는 딕셔너리가 전달됩니다. 또한 다음 속성을 참조할 수 있습니다.

* `self.model`: 콜백을 호출하는 모델 객체
* `self.validation_data`: fit() 메서드에 전달된 검증 데이터

다음은 매 에포크의 끝에서 검증 세트의 첫번째 샘플로 모델에 있는 모든 층의 활성화 출력을 계산하여 디스크에 저장하는 자작 콜백의 예입니다

In [2]:
import keras
import numpy as np

class ActivationLogger(keras.callbacks.Callback):
    
    ## 호출하는 모델에 대한 정보를 전달하기 위해 훈련하기 전에 호출
    def set_model(self, model):
        self.model = model
        layers_outputs = [layer.output for layer in model.layers]
        self.activation_model = keras.models.Model(model.input,
                                                   layers_outputs)
        
    def on_epoch_end(self, epoch, logs = None):
        if self.validation_data is None:
            raise RuntimeError("Requires validation_data")
            
        validation_sample = self.validation_data[0][0:1]
        activations = self.activation_model.predict(validation_sample)
        f = open("activations_at_epochs_" + str(epoch) + ".npz", "wb")
        np.savez(f, activations)
        f.close()                     

### 텐서보드

좋은 연구를 하거나 좋은 모델을 개발하려면 실험하는 모델 내부에서 어떤 일이 일어나는지 자주 그리고 많은 피드백을 받아야 합니다. 그것이 실험을 하는 목적입니다. 모델이 얼마나 잘 작동하는지 가능한 많은 정보를 얻는 것입니다.

텐서보드의 핵심 목적은 훈련 모델의 내부에서 일어나는 모든 것을 시각적으로 모니터링할 수 있도록 돕는 것입니다.
* 모델의 최종 손실 외에 더 많은 정보를 모니터링하면 모델 작동에 대한 명확한 그림을 그릴 수 있습니다
* 결국 모델을 더 빠르게 개선할 수 있습니다

텐서보드로
* 훈련하는 동안 측정 지표를 시각적으로 모니터링
* 모델 구조를 시각화
* 활성화 출력과 그래디언트의 히스토그램을 그림
* 3D로 임베딩을 표현

간단한 예를 사용하여 기능을 실습해보겠습니다.

##### 텐서보드를 사용한 텍스트 분류 모델

In [3]:
import keras
from keras import layers
from keras.datasets import imdb
from keras.preprocessing import sequence

max_features = 2000
max_len = 500

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words = max_features)
x_train = sequence.pad_sequences(x_train, maxlen = max_len)
x_test = sequence.pad_sequences(x_test, maxlen = max_len)

model = keras.models.Sequential()
model.add(layers.Embedding(max_features, 128, 
                           input_length = max_len, name = 'embed'))
model.add(layers.Conv1D(32, 7, activation = "relu"))
model.add(layers.MaxPooling1D(5))
model.add(layers.Conv1D(32, 7, activation = "relu"))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(1))
model.summary()
model.compile(optimizer = "rmsprop",
              loss = "binary_crossentropy",
              metrics = ["acc"])

<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/home/jhhur/anaconda3/envs/TIL/lib/python3.8/site-packages/tensorflow/python/keras/datasets/imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/home/jhhur/anaconda3/envs/TIL/lib/python3.8/site-packages/tensorflow/python/keras/datasets/imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or sha

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embed (Embedding)            (None, 500, 128)          256000    
_________________________________________________________________
conv1d (Conv1D)              (None, 494, 32)           28704     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 98, 32)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 92, 32)            7200      
_________________________________________________________________
global_max_pooling1d (Global (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 291,937
Trainable params: 291,937
Non-trainable params: 0
__________________________________________________

텐서보드를 시작하기 전에 로그파일이 저장될 디렉터리를 만들어야 합니다

In [4]:
!mkdir my_log_dir

mkdir: cannot create directory ‘my_log_dir’: File exists


##### 텐서보드 콜백과 함께 모델 훈련

In [ ]:
callbacks = [
    keras.callbacks.TensorBoard(log_dir = "my_log_dir",
                                histogram_freq = 1,
                                embeddings_freq = 1)
]

history = model.fit(x_train, y_train,
                    epochs = 20,
                    batch_size = 128,
                    validation_split = 0.2,
                    callbacks = callbacks)

Epoch 1/20
157/157 [==============================] - 5s 18ms/step - loss: 0.6566 - acc: 0.6099 - val_loss: 0.4320 - val_acc: 0.8420
Epoch 2/20
157/157 [==============================] - 2s 13ms/step - loss: 0.4142 - acc: 0.8603 - val_loss: 0.4186 - val_acc: 0.8468
Epoch 3/20
157/157 [==============================] - 2s 13ms/step - loss: 0.3607 - acc: 0.8874 - val_loss: 0.4823 - val_acc: 0.8682
Epoch 4/20
157/157 [==============================] - 2s 13ms/step - loss: 0.3393 - acc: 0.9066 - val_loss: 0.5312 - val_acc: 0.8648
Epoch 5/20
157/157 [==============================] - 2s 13ms/step - loss: 0.2880 - acc: 0.9230 - val_loss: 0.6330 - val_acc: 0.8544
Epoch 6/20
157/157 [==============================] - 2s 13ms/step - loss: 0.2563 - acc: 0.9410 - val_loss: 0.5712 - val_acc: 0.8704
Epoch 7/20
157/157 [==============================] - 2s 13ms/step - loss: 0.2028 - acc: 0.9576 - val_loss: 0.6812 - val_acc: 0.8706
Epoch 8/20
 66/157 [===========>..................] - ETA: 1s - loss:

```bash
tensorboard --logdir=my_log_dir
```

위 명령어를 사용하여 텐서보드를 실행 시킬수 있습니다